### Libs

In [ ]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error

### Functions

In [ ]:
def parse_price(val):
    if isinstance(val, str): 
        if "₽" in val:
            val = val.split('₽')[0]
            
        val = val.replace(' ', '')
        return int(val) / 1000000
    
    return float(val)

def parse_area(val):
    if isinstance(val, int): return val
    if isinstance(val, float): return val
    
    return float(val.split("м")[0].replace(" ", ""))

def parse_floor(val):
    if isinstance(val, int): return val
    if isinstance(val, str):
        return val.split('/')[0]
    return val

def get_metro_station(row):
        for i in row:
            if 'МЦК' in i:
                return i
            
def check_log_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()

    X = df_train[feats]
    y = df_train["price"]
    y_log = np.log(y)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_log_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

        model = xgb.XGBRegressor(max_depth=md, n_estimators=ne, learning_rate=lr, random_state=0)
        model.fit(X_train, y_log_train)
        y_log_pred = model.predict(X_test)
        y_pred = np.exp(y_log_pred)

        score = mean_absolute_error(y_test, y_pred)
        scores.append(score)

    return np.mean(scores), np.std(scores)

### Import data

In [3]:
df_train = pd.read_hdf("../input/train_data.h5")
df_train['price'] = df_train['price'].map(parse_price)

df_test = pd.read_hdf("../input/test_data.h5")

df = pd.concat([df_train, df_test])
print(df_train.shape, df_test.shape)

(22732, 8) (11448, 7)


### Main

In [4]:
params = df["params"].apply(pd.Series)
'''
# Checking model performance without features with high percentage of missing values
params_null = params.isnull().sum() / len(params)
missing_features = params_null[params_null > 0.95].index
params.drop(missing_features, axis=1, inplace=True)
'''

params = params.fillna(-1)

if "Охрана:" not in df:
    df = pd.concat([df, params], axis=1)
    
    obj_feats = params.select_dtypes(object).columns

    for feat in obj_feats:
        df["{}_cat".format(feat)] = df[feat].factorize()[0]

cat_feats = [x for x in df.columns if "_cat" in x]

##### Area

In [5]:
df["area"] = df["Общая площадь:"].map(parse_area)

##### Kitchen area

In [6]:
df["kitchen_area"] = df["Площадь кухни:"].map(parse_area)

#### Geo block

In [7]:
geo_block = (
df["geo_block"]
    .map(lambda x: x[:int(len(x)/2) ])
    .map(lambda x: {"geo_block_{}".format(idx):val for idx,val in enumerate(x) })
    .apply(pd.Series)
)

In [8]:
for feat in geo_block.columns:
    df["{}_cat".format(feat)] = geo_block[feat].factorize()[0]

In [9]:
geo_cat_feats = [x for x in df.columns if "geo_block" in x and "_cat" in x]

#### Breadcrumbs

In [10]:
breadcrumbs = (
    df["breadcrumbs"]
        .map(lambda x: {"breadcrumbs_{}".format(idx):val for idx,val in enumerate(x) })
        .apply(pd.Series)
)

In [11]:
for feat in breadcrumbs.columns:
    df["{}_cat".format(feat)] = breadcrumbs[feat].factorize()[0]

In [12]:
breadcrumbs_cat_feats = [x for x in df.columns if "breadcrumbs" in x and "_cat" in x]

#### Floor

In [13]:
df["floor"] = df["Этаж:"].map(parse_floor)
df["floor"] = df["floor"].astype(str).astype(float)

#### Metro station

In [14]:
metro_station = (
    df["breadcrumbs"]
        .map(lambda x: get_metro_station(x))
        .apply(pd.Series)
)
metro_station.columns = ['metro_station_name']

In [15]:
df["metro_station_cat"] = metro_station.apply(lambda x : pd.factorize(x)[0])

In [16]:
df

,geo_block,breadcrumbs,price,owner,params,date,user_block,id,Охрана:,Тип здания:,...,breadcrumbs_0_cat,breadcrumbs_1_cat,breadcrumbs_2_cat,breadcrumbs_3_cat,breadcrumbs_4_cat,breadcrumbs_5_cat,breadcrumbs_6_cat,breadcrumbs_7_cat,floor,metro_station_cat
0,"[г. Москва, Лианозово, г. Москва, Лианозово]","[Москва, Лианозово, м. Алтуфьево, МЦК Лихоборы]",5.402084,[],"{'Охрана:': 'закрытая территория', 'Тип здания...","[22 апреля, 12, (+1 за сегодня), Обновлено 18 ...","[<div class=""block-user__name"">НДВ Недвижимост...",34995,закрытая территория,Монолитное,...,0,0,0,0,-1,-1,-1,-1,8.0,0
1,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",6.827990,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15030,-1,Монолитное,...,0,1,1,1,-1,-1,-1,-1,6.0,1
2,"[г. Москва, ул Лобачевского, г. Москва, ул Лоб...","[Москва, МЦК Кутузовская, ул. Лобачевского]",8.680000,"[ЖК ""Крылья"". Доп. скидка 5%!]","{'Лифт:': 'да', 'Охрана:': 'закрытая территори...","[20 марта, 12, (+2 за сегодня), Обновлено 18 мая]",[],37811,закрытая территория,Монолитное,...,0,2,2,-1,-1,-1,-1,-1,20.0,2
3,"[г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...","[Москва, Ховрино, м. Ховрино, МЦК Коптево, ул....",8.624435,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[29 апреля, 3, (+1 за сегодня), Обновлено 13 мая]","[<div class=""block-user__name"">MR-Group</div>,...",31685,-1,Монолитное,...,0,1,1,1,0,-1,-1,-1,27.0,1
4,"[г. Москва, Даниловский, г. Москва, Даниловский]","[Москва, Даниловский, м. Автозаводская, МЦК Ав...",29.500000,[],"{'Общая площадь:': '141.1 м²', 'Дата публикаци...","[15 декабря 2018, 30, (+1 за сегодня), Обновле...","[<div class=""block-user__name"">Наталья Вдовиче...",56215,-1,Монолитное,...,0,3,3,2,-1,-1,-1,-1,2.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11443,"[г. Москва, г. Москва]","[Москва, МЦК Кутузовская]",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[17 мая, 3, (+2 за сегодня)]","[<div class=""block-user__name"">MR Group</div>,...",47294,предусмотрена,Монолитное,...,0,2,-1,-1,-1,-1,-1,-1,4.0,2
11444,"[г. Москва, Хорошёво-Мнёвники, г. Москва, Хоро...","[Москва, Хорошёво-Мнёвники, м. Октябрьское пол...",NaN,[],"{'Лифт:': 'да', 'Охрана:': 'предусмотрена', 'Т...","[16 мая, 1, (+1 за сегодня)]","[<div class=""block-user__name"">Офис продаж ЖК ...",14091,предусмотрена,Монолитно-кирпичное,...,0,22,22,18,-1,-1,-1,-1,11.0,19
11445,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",NaN,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 1, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",13059,-1,Монолитное,...,0,1,1,1,-1,-1,-1,-1,21.0,1
11446,"[Новая Москва, п. Внуковское, г. Москва, Новая...","[Москва, Новая Москва, п. Внуковское, МЦК Куту...",NaN,[Переделкино Ближнее],"{'Лифт:': 'да', 'Охрана:': 'консьерж', 'Тип зд...","[вчера в 10:14, 2, (+2 за сегодня)]",[],10144,консьерж,Монолитное,...,0,5,21,9,-1,-1,-1,-1,4.0,2


#### ...Lots of hyperparameters values combinations later...

In [17]:
md = 8
ne = 700
lr = 0.1

In [18]:
feats = ["area", "kitchen_area", "floor", "metro_station_cat"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats 
      
check_log_model(df, feats, xgb.XGBRegressor(max_depth=md, n_estimators=ne, learning_rate=lr, random_state=0))

(2.5215593293992624, 0.3107328168425447)

In [21]:
feats = ["area", "kitchen_area", "floor", "metro_station_cat"] + geo_cat_feats + cat_feats + breadcrumbs_cat_feats

df_train = df[ ~df["price"].isnull() ].copy()
df_test = df[ df["price"].isnull() ].copy()

X_train = df_train[feats]
y_train = df_train["price"]
y_log_train = np.log(y_train)

X_test = df_test[feats]

model = xgb.XGBRegressor(max_depth=8, n_estimators=700, learning_rate=0.1, random_state=0)
model.fit(X_train, y_log_train)
y_log_pred = model.predict(X_test)
y_pred = np.exp(y_log_pred)


df_test["price"] = y_pred
df_test[ ["id", "price"] ].to_csv("../output/xgb_location_log_area_v2.csv", index=False)